In [1]:
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
import numpy as np 

In [27]:
def read_skeleton_file(filename): # อ่านไฟล์ .skeleton
    with open(filename, 'r') as f:
        lines = f.readlines()

    index = 0
    num_frames = int(lines[index].strip()); index += 1
    data = []

    for _ in range(num_frames):
        frame = {}
        num_bodies = int(lines[index].strip()); index += 1
        frame['bodies'] = []

        for _ in range(num_bodies):
            body_id_line = lines[index].strip(); index += 1
            while not lines[index].strip().isdigit():
                index += 1

            num_joints = int(lines[index].strip()); index += 1
            joints = []

            for _ in range(num_joints):
                parts = lines[index].strip().split()
                x, y, z = map(float, parts[:3])
                joints.append((x, y, z))
                index += 1

            frame['bodies'].append({'joints': joints})

        data.append(frame)

    return data

In [28]:
def get_sequence_length(skeleton_data): # ดึงจำนวนเฟรมทั้งหมด
    return len(skeleton_data)

In [29]:
def extract_joint_sequence(skeleton_data, joint_id, body_index=0): # ดึงตำแหน่งของ joint ที่กำหนดตลอด sequence
    sequence = []
    for frame in skeleton_data:
        try:
            joints = frame['bodies'][body_index]['joints']
            if joint_id < len(joints):
                sequence.append(joints[joint_id])
        except:
            continue
    return sequence

In [30]:
def compute_movement(joint_sequence): # คำนวณระยะการเคลื่อนไหว รวม เฉลี่ย และ SD ของข้อมือขวา
    diffs = []
    for i in range(1, len(joint_sequence)):
        prev = np.array(joint_sequence[i - 1])
        curr = np.array(joint_sequence[i])
        diff = np.linalg.norm(curr - prev)
        diffs.append(diff)
    return sum(diffs), np.mean(diffs), np.std(diffs)

In [31]:
def print_basic_stats(skeleton_data): # แสดงสถิติพื้นฐานเช่น จำนวนเฟรม จำนวนร่างในแต่ละเฟรม
    num_frames = len(skeleton_data)
    max_bodies = max(len(f) for f in skeleton_data)
    print(f"Total frames: {num_frames}")
    print(f"Max bodies in a frame: {max_bodies}")

In [33]:
if __name__ == '__main__':
    filename = "/Users/balast/Desktop/LiftingProject/LiftingDetection/ActionRecognition/notebooks/DataTest/S001C001P001R001A001.skeleton"
    skeleton_data = read_skeleton_file(filename)

    print_basic_stats(skeleton_data)

    length = get_sequence_length(skeleton_data)
    print(f"Sequence Length: {length} frames")

    right_wrist_seq = extract_joint_sequence(skeleton_data, joint_id=15)
    print("Right wrist positions (first few):", right_wrist_seq[:5])

    total, mean, std = compute_movement(right_wrist_seq)
    print(f"Right wrist movement: total={total:.3f}, mean={mean:.3f}, std={std:.3f}")

    frame_idx = 1
    for idx, frame in enumerate(skeleton_data[:frame_idx]):
        print(f"\nFrame {idx + 1}:")
        for body in frame['bodies']:
            for joint_id, (x, y, z) in enumerate(body['joints']):
                print(f"  Joint {joint_id}: x={x:.3f}, y={y:.3f}, z={z:.3f}")

Total frames: 103
Max bodies in a frame: 1
Sequence Length: 103 frames
Right wrist positions (first few): [(0.2040377, -0.476396, 4.094294), (0.1938665, -0.4768415, 3.967519), (0.1940295, -0.4770178, 3.970365), (0.2325329, -0.4818147, 4.126212), (0.1945294, -0.4784354, 3.967172)]
Right wrist movement: total=3.289, mean=0.032, std=0.051

Frame 1:
  Joint 0: x=0.218, y=0.173, z=3.786
  Joint 1: x=0.232, y=0.433, z=3.715
  Joint 2: x=0.246, y=0.688, z=3.634
  Joint 3: x=0.213, y=0.808, z=3.582
  Joint 4: x=0.111, y=0.611, z=3.717
  Joint 5: x=0.101, y=0.429, z=3.743
  Joint 6: x=0.119, y=0.343, z=3.557
  Joint 7: x=0.117, y=0.317, z=3.551
  Joint 8: x=0.345, y=0.574, z=3.589
  Joint 9: x=0.421, y=0.385, z=3.628
  Joint 10: x=0.192, y=0.319, z=3.541
  Joint 11: x=0.130, y=0.317, z=3.543
  Joint 12: x=0.163, y=0.175, z=3.778
  Joint 13: x=0.198, y=-0.136, z=3.904
  Joint 14: x=0.244, y=-0.420, z=4.054
  Joint 15: x=0.204, y=-0.476, z=4.094
  Joint 16: x=0.270, y=0.168, z=3.730
  Joint 17: x